<h3>Toy example experiment 6.18</h3>

Use the magenpy GWASimulator based on RA's genotypes data (d1CM22_sorted.bed) to simulate phenotypes.  
the heritibility = 0.5, (Gaussian Mixture Proportion)pi = [.85, .15]

In [1]:
import numpy as np
import pandas as pd
import magenpy as mgp
import viprs as vp

In [2]:
#simulate 
g_sim = mgp.GWASimulator("CMdream1/chr22/d1CM22_sorted",
                         pi = [.99, .01],
                         h2=0.5)
g_sim.simulate()
g_sim.to_phenotype_table()

> Reading BED file...
> Generating polygenic scores...


,FID,IID,phenotype
0,0,1721000,-1.010706
1,0,1722000,0.865608
2,0,1723000,0.122068
3,0,1725000,0.454301
4,0,1726000,0.268947
...,...,...,...
172,0,1921000,-1.507169
173,0,1922000,1.224331
174,0,1923000,-1.111359
175,0,1924000,-1.780030


In [3]:
pd.read_csv("CMdream1/chr22/d1CM22_sorted.bim", sep="\t", header=None)


,0,1,2,3,4,5
0,22,rs4911642,-11.277499,14884399,G,A
1,22,rs2027653,-7.277181,15298335,G,A
2,22,rs5747620,-6.171966,15412698,G,A
3,22,rs5747968,-5.835598,15447504,C,A
4,22,rs2236639,-5.787480,15452483,A,G
...,...,...,...,...,...,...
7915,22,rs715586,70.074142,49510004,A,G
7916,22,rs8137951,70.076278,49512530,A,G
7917,22,rs756638,70.084762,49518559,A,G
7918,22,rs3810648,70.168330,49522492,G,A


In [4]:
g_sim.perform_gwas()
g_sim.to_summary_statistics_table().to_csv(
    "Toy_example_expr/d1CM22_sorted.sumstats", sep="\t", index=False
)

> Performing GWAS...


In [5]:
# Load summary statistics（simulate phenotype from above) and match them with perviously
gdl_sim = mgp.GWADataLoader(bed_files="CMdream1/chr22/d1CM22_sorted",
                            sumstats_files="Toy_example_expr/d1CM22_sorted.sumstats",
                            sumstats_format="magenpy")

> Reading BED file...
> Reading summary statistics file...
> Harmonizing data...


In [6]:
pd.read_csv("Toy_example_expr/d1CM22_sorted.sumstats", sep="\t", header=None)

,0,1,2,3,4,5,6,7,8,9,10
0,CHR,SNP,POS,A1,A2,MAF,N,BETA,Z,SE,PVAL
1,22,rs4911642,14884399,G,A,0.12721893491124261,169,0.06405049344662028,0.8326564148060637,0.07692307692307693,0.405038532309158
2,22,rs2027653,15298335,G,A,0.3742690058479532,171,-0.015638625018954477,-0.20450155822064825,0.07647191129018725,0.83796157444101
3,22,rs5747620,15412698,G,A,0.3953488372093023,172,-0.09178566334704852,-1.2037576896211002,0.07624928516630233,0.22868324742188484
4,22,rs5747968,15447504,C,A,0.3700564971751412,177,-0.042020555905944904,-0.5590471357587852,0.07516460280028288,0.5761295517091787
...,...,...,...,...,...,...,...,...,...,...,...
7916,22,rs715586,49510004,A,G,0.14204545454545456,176,0.070783797882582,0.9390531952515101,0.07537783614444091,0.3477034332638703
7917,22,rs8137951,49512530,A,G,0.2768361581920904,177,-0.039066507371861674,-0.5197460761638542,0.07516460280028288,0.6032405678882882
7918,22,rs756638,49518559,A,G,0.2471590909090909,176,0.1017155631818596,1.3494094336556661,0.07537783614444091,0.1772054923890033
7919,22,rs3810648,49522492,G,A,0.048295454545454544,176,0.12056290315911831,1.5994476536589965,0.07537783614444091,0.1097211709879028


In [7]:
# gdl_sim.compute_ld(estimator='sample',
#                    output_dir='Toy_example_expr/out/ld/')
gdl_sim.compute_ld(estimator='windowed',
                   output_dir='Toy_example_expr/out/ld/',
                   window_size=100)

> Computing LD matrix...


In [8]:
# attempt to read the computed LD
gdl_sim.load_ld()
gdl_sim.ld

{22: <magenpy.LDMatrix.LDMatrix at 0x7f95306180b0>}

In [9]:
# i =  10
# d1_LD = mgp.LDMatrix.from_path("Toy_example_expr/out/ld/ld/chr_22")
# # for i in range(0,7920):
# #     print(d1_LD[i].shape)

# d1_LD[2]

Use Viprs, initialize the viprs model based on the <code> gdl_sim model</code> containing the GWAS data and the LD matrix (using xarray method) for the chr22.

In [10]:
v = vp.VIPRS(gdl_sim)

In [11]:
v.fit()

> Initializing model parameters
> Loading LD matrices into memory...
> Performing model fit...
> Using up to 1 threads.


  0%|          | 4/1000 [00:00<00:38, 26.06it/s]

OptimizationDivergence: Stopping at iteration 5: The optimization algorithm is not converging!
Previous ELBO: -220.524828 | Current ELBO: 155017871180485.656250

In [ ]:
# Once the model converges, we can generate PRS estimates 
prs_chr22 = v.predict()
print(len(prs_chr22))
prs_chr22

> Generating polygenic scores...
177


array([-0.04869287, -0.05159315, -0.03341141, -0.04092102,  0.00800504,
       -0.07446621, -0.02891326,  0.07615968, -0.00190596, -0.00153135,
       -0.01466533,  0.00809267, -0.04965925,  0.08105793, -0.07767217,
        0.11093063, -0.00467449,  0.04714662, -0.02972149, -0.03469132,
       -0.03975939, -0.02460893,  0.06840094,  0.04667084, -0.04166952,
        0.03044004, -0.04885707,  0.05560123, -0.04475551, -0.01026837,
       -0.05393578,  0.0804356 , -0.02755772,  0.02766405, -0.02760438,
        0.03990991,  0.01237181,  0.10902726,  0.02366103,  0.0189675 ,
        0.04046421, -0.11356237, -0.04550571, -0.00087849, -0.04122994,
        0.07447408, -0.11099115, -0.06686776,  0.01833978, -0.02180401,
       -0.00475278, -0.10805256, -0.07335433,  0.13263711, -0.03423287,
        0.02571957,  0.0739412 , -0.02307223, -0.01119488, -0.02195915,
        0.01482855,  0.0877863 , -0.02144691,  0.06323041, -0.04707012,
       -0.00137178, -0.01651207,  0.05696975, -0.00324996, -0.03

In [ ]:
# To examine posterior estimates for the model parameters, you can simply invoke the .to_table() method:
v.to_table()

,CHR,SNP,A1,A2,BETA,PIP,VAR_BETA
0,22,rs4911642,G,A,-0.000205,0.050554,0.000011
1,22,rs2027653,G,A,-0.000277,0.052010,0.000012
2,22,rs5747620,G,A,0.000092,0.048995,0.000010
3,22,rs5747968,C,A,-0.000058,0.048725,0.000010
4,22,rs2236639,A,G,0.000131,0.049394,0.000010
...,...,...,...,...,...,...,...
7915,22,rs715586,A,G,-0.000012,0.048570,0.000010
7916,22,rs8137951,A,G,0.000035,0.048620,0.000010
7917,22,rs756638,A,G,0.000100,0.049048,0.000010
7918,22,rs3810648,G,A,0.000134,0.049437,0.000010


<code>Validation</code> use another the simulate phenotype still on the d1chr22_sorted.bed as the validation part. 

In [ ]:
#simulate 
g2_sim = mgp.GWASimulator("CMdream1/chr22/d1CM22_sorted",
                         pi = [.85, .15],
                         h2=0.5)
g2_sim.simulate()
a = g2_sim.to_phenotype_table()
a.to_csv("Toy_example_expr/phenotype/g2_sim_pheno.txt",sep='\t', index=False, header=False)
g2_sim.perform_gwas()
g2_sim.to_summary_statistics_table().to_csv(
    "Toy_example_expr/d1CM22_sorted_valid.sumstats", sep="\t", index=False
)

# Load summary statistics（simulate phenotype from above) and match them with perviously
gdl2_sim = mgp.GWADataLoader(bed_files="CMdream1/chr22/d1CM22_sorted",
                            phenotype_file="Toy_example_expr/phenotype/g2_sim_pheno.txt",
                            sumstats_files="Toy_example_expr/d1CM22_sorted_valid.sumstats",
                            sumstats_format="magenpy")
gdl2_sim.compute_ld(estimator='sample',
                   output_dir='Toy_example_expr/out/ld_val/')


> Reading BED file...
> Generating polygenic scores...
> Performing GWAS...


> Reading BED file...
> Reading phenotype file...
> Reading summary statistics file...
> Harmonizing data...
> Computing LD matrix...


In [ ]:
val_prs = v.predict(gdl2_sim)

> Generating polygenic scores...


In [ ]:
from viprs.eval.metrics import r2 
r2(val_prs, gdl2_sim.sample_table.phenotype)

0.0030078829752389377

Validation use the chr22 genotypes in <code> dream2 </code> file sets.  
Do the same steps as before to got <code>"CMdream2/chr22/d2CM22_sorted"</code> binary file sets (.bed + .fam + .bim).  
Sorted the SNPs from dream2 for chromosome 22 and add CM values to it. 

In [ ]:
d2_bim = pd.read_csv("data/RA_challenge_qced_genotypes/dream2.bim", sep="\t", header=None)
d2_filter_bim = d2_bim[d2_bim[0] ==22][1]
d2_sorted_chr22 = d2_filter_bim.sort_values()
d2_sorted_chr22.to_csv("CMdream2/d2_sorted_chr22_bim.txt", header=False,
                       index=False, sep="\t")

In [ ]:
# Use simulate method to create phenotpyes 
#simulate 
g3_sim = mgp.GWASimulator("CMdream2/chr22/d2CM22_sorted",
                         pi = [.85, .15],
                         h2=0.5)
g3_sim.simulate()
a = g3_sim.to_phenotype_table()
a.to_csv("Toy_example_expr/phenotype/val_d2chr22_pheno.txt",sep='\t', index=False, header=False)
g3_sim.perform_gwas()
g3_sim.to_summary_statistics_table().to_csv(
    "Toy_example_expr/val_d2chr22_valid.sumstats", sep="\t", index=False
)

# Load summary statistics（simulate phenotype from above) and match them with perviously
gdl3_sim = mgp.GWADataLoader(bed_files="CMdream2/chr22/d2CM22_sorted",
                            phenotype_file="Toy_example_expr/phenotype/val_d2chr22_pheno.txt",
                            sumstats_files="Toy_example_expr/val_d2chr22_valid.sumstats",
                            sumstats_format="magenpy")
gdl3_sim.compute_ld(estimator='sample',
                   output_dir='Toy_example_expr/d2chr22_out/ld_val/')

> Reading BED file...
> Generating polygenic scores...
> Performing GWAS...
> Reading BED file...
> Reading phenotype file...
> Reading summary statistics file...
> Harmonizing data...
> Computing LD matrix...


Exception ignored in: <finalize object at 0x7fd075eb7030; dead>
Traceback (most recent call last):
  File "/usr/lib/python3.7/weakref.py", line 572, in __call__
    return info.func(*info.args, **(info.kwargs or {}))
  File "/usr/lib/python3.7/tempfile.py", line 934, in _cleanup
    _rmtree(name)
  File "/usr/lib/python3.7/shutil.py", line 485, in rmtree
    onerror(os.lstat, path, sys.exc_info())
  File "/usr/lib/python3.7/shutil.py", line 483, in rmtree
    orig_st = os.lstat(path)
FileNotFoundError: [Errno 2] No such file or directory: 'temp/ld_aqvxu2v_'


In [ ]:
val2_prs = v.predict(gdl3_sim)

> Generating polygenic scores...


In [ ]:
#To evaluate the accuracy of the polygenic score, 
# you can use some of the provided metrics with the viprs package. 
# For quantitative traits, it is customary to report the prediction R^2, 
# or the proportion of variance explained by the PRS:
r2(val2_prs, gdl3_sim.sample_table.phenotype)

0.011801941348497371

In [ ]:
#simulate 
g2_sim = mgp.GWASimulator("data/RA_challenge_qced_genotypes/dream3",
                         pi = [.85, .15],
                         h2=0.5)
g2_sim.simulate()
a = g2_sim.to_phenotype_table()

> Reading BED file...


Generating polygenic scores: 100%|██████████| 27/27 [00:06<00:00,  4.33it/s]


In [ ]:
a

,FID,IID,phenotype
0,0,2280000,-1.159332
1,0,2282000,0.457723
2,0,2283000,0.709957
3,0,2284000,-0.482596
4,0,2285000,0.660879
...,...,...,...
412,0,2767000,-1.817492
413,0,2768000,-0.738611
414,0,2769000,-0.103463
415,0,2770000,-1.178744


<h3>June 24th repeat above experiment with the ALL_CM_chr22 data</h3>
We checked dream1,2,3 binary files they contain <code>880</code> patients in total. To get the <code>2706 patients</code> we need to <code>merge all</code> RA_challenge_qced_genotypes data together. 

We used the command below to created merge files:  
<code>cd data/RA_challenge_qced_genotypes && bash ../utils/mergeSNPS.sh</code>  
The last file <code> merge10 </code> is the complete file that contains all 11 experiments data, which sums up to 2706 patients.  

Therefore, now we use the <code> merge10.bed + .bim + .fam </code> files to repeat our experiments with magenpy and viprs use the simulator. 



In [1]:
import numpy as np
import pandas as pd
import magenpy as mgp
import viprs as vp

In [3]:
#simulate 
g_sim = mgp.GWASimulator("CMAll_qced/chr22/ALL_CM_chr22",
                         pi = [.99, .01],
                         h2=0.5)
g_sim.simulate()
g_sim.to_phenotype_table()

> Reading BED file...
> Generating polygenic scores...


,FID,IID,phenotype
0,0,1000,0.117538
1,0,2000,-0.186721
2,0,3000,-0.929450
3,0,4000,0.151187
4,0,5000,-0.076602
...,...,...,...
2701,0,3061000,-0.432134
2702,0,3062000,0.094320
2703,0,3064000,-0.632079
2704,0,3065000,-0.628738


In [4]:
g_sim.perform_gwas()
g_sim.to_summary_statistics_table().to_csv(
    "Toy_example_expr/ALL_CM22.sumstats", sep="\t", index=False
)

> Performing GWAS...


/home/mcb/users/yzhu439/RA_Project/.venv/lib/python3.7/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))


In [5]:
# Load summary statistics（simulate phenotype from above) and match them with perviously
gdl_sim = mgp.GWADataLoader(bed_files="CMAll_qced/chr22/ALL_CM_chr22",
                            sumstats_files="Toy_example_expr/ALL_CM22.sumstats",
                            sumstats_format="magenpy")

> Reading BED file...
> Reading summary statistics file...
> Harmonizing data...


In [6]:
gdl_sim.compute_ld(estimator='sample',
                   output_dir='Toy_example_expr/ALLchr22_out/')
# gdl_sim.compute_ld(estimator='windowed',
#                    output_dir='Toy_example_expr/out/ld/',
#                    window_size=100)

> Computing LD matrix...


/home/mcb/users/yzhu439/RA_Project/.venv/lib/python3.7/site-packages/dask/array/routines.py:318: PerformanceWarning: Increasing number of chunks by factor of 18
  axes=(left_axes, right_axes),
/home/mcb/users/yzhu439/RA_Project/.venv/lib/python3.7/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))


Use Viprs, initialize the viprs model based on the <code> gdl_sim model</code> containing the GWAS data and the LD matrix (using xarray method) for the chr22 for all qced dataset (with 2706 individuals).

In [7]:
v = vp.VIPRS(gdl_sim)

In [9]:
v.fit()

> Initializing model parameters
> Loading LD matrices into memory...
> Performing model fit...
> Using up to 1 threads.


 37%|███▋      | 367/1000 [03:35<06:11,  1.70it/s]


OptimizationDivergence: Stopping at iteration 368: The optimization algorithm is not converging!
Previous ELBO: -2997.439389 | Current ELBO: 713017877637161.125000